# Use FairMedFM to perform linear probing

In [1]:
!git clone https://github.com/FairMedFM/FairMedFM

Cloning into 'FairMedFM'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (324/324), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 324 (delta 174), reused 233 (delta 95), pack-reused 0
Receiving objects: 100% (324/324), 2.89 MiB | 10.98 MiB/s, done.
Resolving deltas: 100% (174/174), done.


In [5]:
%cd FairMedFM

/content/FairMedFM


We use HAM10000 as an example for demonstration.

In [3]:
%mkdir data
%cd data
!wget https://object-arbutus.cloud.computecanada.ca/rjin/HAM10000.zip
!unzip HAM10000.zip
%cd ..

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: HAM10000/HAM10000_images/ISIC_0029322.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029323.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029324.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029325.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029326.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029327.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029328.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029329.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029330.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029331.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029332.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029333.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029334.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029335.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029336.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029337.jpg  
  inflating: HAM10000/HAM10000_images/ISIC_0029338.jpg  
  in

In [15]:
!pip install icecream==2.1.3
!pip install SimpleITK==2.3.1
!pip install einops==0.8.0
!pip install ipdb==0.13.13
!pip install git+https://github.com/facebookresearch/segment-anything.git@6fdee8f2727f4506cfbbe553e23b895e2795658
!pip install timm==1.0.3
!pip install open-clip-torch==2.24.0
!pip install salesforce-lavis==1.0.2
!pip install git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33
!pip install MedCLIP==0.0.3
!pip install torchmetrics==1.4.0.post0
!pip install peft==0.11.1
!pip install git+https://github.com/transcendentsky/tutils.git@8b02b20c9caa5553c96a6413b739ac47385fd730
!pip install transformers==4.40.2

  Cloning https://github.com/facebookresearch/segment-anything.git (to revision 6fdee8f2727f4506cfbbe553e23b895e2795658) to /tmp/pip-req-build-qd_jhgio
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-qd_jhgio
  Running command git checkout -q 6fdee8f2727f4506cfbbe553e23b895e2795658
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e2795658
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=39b382617f206a86f9208e519b61b03c59865e28a464f7052391da84fc91e1e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-h3nopj8j/wheels/ed/f9/15/7c73be3b9a46fe7a94380213fa6e09fb52961ee583b31d6c3c
Successfully built segment-anything


In [35]:
!pip freeze

absl-py==1.4.0
accelerate==0.32.1
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
asttokens==2.4.1
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.11.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
braceexpand==0.1.7
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.4.0
catalogue==2.0.10
certifi==2024.7.4
cffi==1.16.0
cfgv==3.4.0
chardet==3.0.4
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
clip @ git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33
cloudpathlib==0.18.1
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.4
colorama==0.4.6
colorce

Let's use BioMedCLIP for linear probing.

In [36]:
!CUDA_VISIBLE_DEVICES=0 python main.py --task cls --usage lp --dataset HAM10000 --model BiomedCLIP --sensitive_name Sex --method erm --total_epochs 50 --warmup_epochs 5 --blr 2.5e-4 --batch_size 128 --optimizer adamw --min_lr 1e-5 --weight_decay 0.05


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_5m_224 in registry with models.sam_builder.build_tinysam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_11m